In [ ]:
#catch output of func and return it
def silent(func, *args):
  from contextlib import redirect_stdout
  import io
  ret = None
  
  with redirect_stdout(io.StringIO()) as out:
      ret = func(*args)
  return (out.getvalue(), ret)

# executes <func> for all DataFrames in a dict and prints the output of <func> 
def explore_all(dict_of_frames, func):
  from types import MethodType

  #test func type
  if not isinstance(func, str):
    raise TypeError(f"explore_all error: func needs to be a str of function " \
      f"name without() not {type(func)}")
    
  #test if dict is really a dict
  if not isinstance(dict_of_frames, dict):
    raise TypeError(f"explore_all error: dict_of_frames needs to be a dict not" \
      f" {type(dict_of_frames)}")
  else:
    #test if the dict contains DataFrames or Series
    testitem = next(iter(dict_of_frames.values()))
    if not isinstance(testitem, pd.DataFrame) \
            and not isinstance(testitem, pd.Series):
      raise TypeError(f"explore_all error: dict_of_frames needs to be a dict of" \
        f" DataFrames or Series not {type(testitem)}")

    # test if dict can execute the func
    if not hasattr(testitem, func):
      raise AttributeError(f"explore_all error: {type(testitem)} has no" \
        f" method {func}()")

  #finally printing what we want to see
  print(f'{func:-^30}')
  for k in dict_of_frames.keys():
    print(f'{k:.^30}')
    obj = getattr(dict_of_frames[k], func)
    if isinstance(obj, MethodType):
      ret = obj()
      if not ret is None:
        print(ret) #prints methods that return someting but not those that print directly
    else:
      print(obj) #prints attributes
    print()

#paste the output of two outputs beside each other
def explore_all2(dict_of_frames, func1, func2):
  out1, _ = silent(explore_all, dict_of_frames, func1)
  out2, _ = silent(explore_all, dict_of_frames, func2)
  
  out1, out2 = align_2texts([out1, out2])
  out1_maxwidth = max([len(x) for x in out1])

  #putting it all together
  list1 = iter(out1)
  list2 = iter(out2)
  for _ in range(len(out1)):
    print(f'{next(list1):{out1_maxwidth}}\t\t{next(list2)}')

def align_2texts(texts):
  texts = [t.split('\n') for t in texts]
  lens = [len(t) for t in texts]
  list_max = max(lens)
  
  if lens[0] == list_max:
    #left side is longer than right side
    texts[1] = pad_right_text(texts,  lens)
  else: 
    #right side is longer than left side
    texts[0] = pad_left_text(texts, lens)

  return texts[0], texts[1]

def pad_right_text(texts, lens):
    return pad_left_text([texts[1], texts[0]], lens)

def pad_left_text(texts, lens):
  text_l = texts[0]
  text_r = texts[1]

  #section_indexes, i.e. ".....brands....."
  sec_idx_l = [i for i,o in enumerate(text_l) if o.startswith('.') and o.endswith('.')] 
  sec_idx_r = [i for i,o in enumerate(text_r) if o.startswith('.') and o.endswith('.')] 
  #start with list of empty strings
  padded_text = [''] * max(lens)
  len_l = min(lens)

  #first line = name of applied func                       
  padded_text[0] = text_l[0]
  
  #fill lines with chunks of text from left text
  #take the list index where to insert the text section from right text
  for i in range(len(sec_idx_r) -1):
    idx_diff = sec_idx_r[i] + sec_idx_l[i+1] - sec_idx_l[i]
    padded_text[sec_idx_r[i]:idx_diff] = text_l[sec_idx_l[i]:sec_idx_l[i+1]]
  
  idx_diff = sec_idx_r[-1] + len_l - sec_idx_l[-1]
  padded_text[sec_idx_r[-1]:idx_diff] = text_l[sec_idx_l[-1]:len_l]

  return padded_text       

#return list of non-numerical columns of a DataFrame
def non_numerical_cols(df):
  num_cols = df.describe().columns
  return [col for col in df.columns if col not in num_cols]